# Gender Wage Gap Analysis: Validated Dataset

This notebook analyzes the **cleaned and validated** dataset that includes:
- **3 Balkan countries**: North Macedonia, Serbia, Montenegro (unreliable data removed)
- **9 EU countries**: Sweden, Italy, Poland, Hungary, Slovenia, Croatia, Romania, Bulgaria, Greece
- **14 World Bank indicators** for labor market context
- **Reliability ratings** for data quality transparency

### Data Quality
- **OFFICIAL**: 59% - Eurostat, National Statistical Offices
- **RESEARCH**: 18% - Academic studies with citations
- **ESTIMATE**: 23% - Sample data, interpolations

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print('Libraries loaded successfully!')

In [ ]:
# Load the VALIDATED integrated dataset
df = pd.read_csv('../data/processed/integrated_wage_data_validated.csv')

print('='*80)
print('VALIDATED DATASET LOADED')
print('='*80)
print(f'\nTotal records: {len(df)}')
print(f'Countries: {len(df["country"].unique())}')
print(f'Columns: {len(df.columns)}')
print(f'\nYears covered: {df["year"].min()} - {df["year"].max()}')

print('\nData Quality Breakdown:')
print(df['reliability'].value_counts())

In [ ]:
# Define regions for analysis
BALKANS = ['North Macedonia', 'Serbia', 'Montenegro']
EU_COUNTRIES = ['Sweden', 'Italy', 'Poland', 'Hungary', 'Slovenia', 
                'Croatia', 'Romania', 'Bulgaria', 'Greece']

# Add region column
df['region'] = df['country'].apply(lambda x: 'Balkans' if x in BALKANS else 'EU')

print('Countries by Region:')
print('-'*40)
print(f'BALKANS ({len(BALKANS)}): {BALKANS}')
print(f'EU ({len(EU_COUNTRIES)}): {EU_COUNTRIES}')

In [ ]:
# World Bank indicator descriptions
WB_INDICATORS = {
    'SE.TER.CUAT.BA.FE.ZS': 'Education: Bachelor+ (Female %)',
    'SE.TER.CUAT.BA.MA.ZS': 'Education: Bachelor+ (Male %)',
    'SL.AGR.EMPL.FE.ZS': 'Employment in Agriculture (Female %)',
    'SL.EMP.WORK.FE.ZS': 'Wage/Salaried Workers (Female %)',
    'SL.EMP.WORK.MA.ZS': 'Wage/Salaried Workers (Male %)',
    'SL.IND.EMPL.FE.ZS': 'Employment in Industry (Female %)',
    'SL.SRV.EMPL.FE.ZS': 'Employment in Services (Female %)',
    'SL.TLF.CACT.FE.ZS': 'Labor Force Participation (Female %)',
    'SL.TLF.CACT.MA.ZS': 'Labor Force Participation (Male %)',
    'SL.TLF.CACT.ZS': 'Labor Force Participation (Total %)',
    'SL.TLF.PART.FE.ZS': 'Part-time Employment (Female %)',
    'SL.TLF.PART.MA.ZS': 'Part-time Employment (Male %)',
    'SL.UEM.TOTL.FE.ZS': 'Unemployment (Female %)',
    'SL.UEM.TOTL.MA.ZS': 'Unemployment (Male %)'
}

print('World Bank Indicators Available:')
for code, name in WB_INDICATORS.items():
    coverage = df[code].notna().sum() / len(df) * 100
    print(f'  {name}: {coverage:.0f}% coverage')

## 2. Gender Pay Gap Overview

In [ ]:
# Gender pay gap ranking by country
print('='*80)
print('GENDER PAY GAP RANKING (Validated Data)')
print('='*80)

ranking = df.groupby('country').agg({
    'wage_gap_pct': 'mean',
    'reliability': lambda x: x.mode()[0],
    'region': 'first'
}).round(1).sort_values('wage_gap_pct', ascending=False)

ranking.columns = ['Wage Gap %', 'Data Quality', 'Region']
print(ranking)

print('\n' + '-'*80)
print('KEY INSIGHTS:')
highest = ranking.index[0]
lowest = ranking.index[-1]
print(f'  Highest gap: {highest} ({ranking.loc[highest, "Wage Gap %"]}%)')
print(f'  Lowest gap: {lowest} ({ranking.loc[lowest, "Wage Gap %"]}%)')
print(f'  Range: {ranking["Wage Gap %"].max() - ranking["Wage Gap %"].min():.1f} percentage points')

In [ ]:
# Visualize wage gap by country with data quality indicators
fig, ax = plt.subplots(figsize=(14, 7))

# Prepare data
gap_data = df.groupby('country').agg({
    'wage_gap_pct': 'mean',
    'reliability': lambda x: x.mode()[0],
    'region': 'first'
}).sort_values('wage_gap_pct', ascending=True)

# Color by region
colors = ['#e74c3c' if r == 'Balkans' else '#3498db' for r in gap_data['region']]

# Edge color by reliability
edge_colors = []
for rel in gap_data['reliability']:
    if rel == 'OFFICIAL':
        edge_colors.append('#27ae60')  # Green
    elif rel == 'RESEARCH':
        edge_colors.append('#f39c12')  # Orange
    else:
        edge_colors.append('#95a5a6')  # Gray

bars = ax.barh(gap_data.index, gap_data['wage_gap_pct'], color=colors, 
               edgecolor=edge_colors, linewidth=3, alpha=0.8)

# Add EU average line
eu_avg = 12.0  # Official Eurostat 2023
ax.axvline(x=eu_avg, color='green', linestyle='--', linewidth=2, 
           label=f'EU Average 2023 ({eu_avg}%)')

# Labels
ax.set_xlabel('Gender Pay Gap (%)', fontsize=12)
ax.set_title('Gender Pay Gap by Country (Validated Data)', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')

# Add value labels
for bar, val, rel in zip(bars, gap_data['wage_gap_pct'], gap_data['reliability']):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, 
            f'{val:.1f}% ({rel[:3]})', va='center', fontsize=10)

# Add legend for colors
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#e74c3c', label='Balkans'),
    Patch(facecolor='#3498db', label='EU Countries'),
    Patch(facecolor='white', edgecolor='#27ae60', linewidth=2, label='Official Data'),
    Patch(facecolor='white', edgecolor='#f39c12', linewidth=2, label='Research Data'),
    Patch(facecolor='white', edgecolor='#95a5a6', linewidth=2, label='Estimate')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=9)

plt.tight_layout()
plt.show()

## 3. Balkans vs EU Comparison

In [ ]:
# Regional comparison
print('='*80)
print('BALKANS vs EU COMPARISON')
print('='*80)

regional = df.groupby('region').agg({
    'wage_gap_pct': ['mean', 'std', 'min', 'max'],
    'country': 'nunique',
    'reliability': lambda x: (x == 'OFFICIAL').sum() / len(x) * 100
}).round(1)

regional.columns = ['Avg Gap %', 'Std Dev', 'Min Gap', 'Max Gap', 'Countries', 'Official Data %']
print(regional)

# Statistical test
balkans_gaps = df[df['region'] == 'Balkans']['wage_gap_pct']
eu_gaps = df[df['region'] == 'EU']['wage_gap_pct']

t_stat, p_value = stats.ttest_ind(balkans_gaps, eu_gaps)

print('\n' + '-'*80)
print('STATISTICAL TEST: Is the difference significant?')
print(f'  Balkans avg: {balkans_gaps.mean():.1f}%')
print(f'  EU avg: {eu_gaps.mean():.1f}%')
print(f'  Difference: {balkans_gaps.mean() - eu_gaps.mean():.1f} percentage points')
print(f'  T-statistic: {t_stat:.3f}')
print(f'  P-value: {p_value:.4f}')
print(f'  Significant at 0.05? {"YES" if p_value < 0.05 else "NO"}')

In [ ]:
# Visualization: Balkans vs EU
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Box plot comparison
df.boxplot(column='wage_gap_pct', by='region', ax=axes[0])
axes[0].set_title('Wage Gap Distribution by Region', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Region')
axes[0].set_ylabel('Wage Gap (%)')
plt.suptitle('')  # Remove automatic title

# 2. Bar comparison with error bars
regions = ['Balkans', 'EU']
means = [balkans_gaps.mean(), eu_gaps.mean()]
stds = [balkans_gaps.std(), eu_gaps.std()]
colors = ['#e74c3c', '#3498db']

bars = axes[1].bar(regions, means, yerr=stds, capsize=5, color=colors, alpha=0.8)
axes[1].axhline(y=12.0, color='green', linestyle='--', label='EU Official Avg (12%)')
axes[1].set_ylabel('Wage Gap (%)')
axes[1].set_title('Average Wage Gap: Balkans vs EU', fontsize=12, fontweight='bold')
axes[1].legend()

# Add value labels
for bar, mean in zip(bars, means):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                 f'{mean:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'\nKey Finding: Balkans wage gap is {balkans_gaps.mean() - eu_gaps.mean():.1f} pp higher than EU average')

## 4. Data Quality Analysis

In [ ]:
# Data quality breakdown
print('='*80)
print('DATA QUALITY BREAKDOWN')
print('='*80)

quality = df.groupby(['region', 'reliability']).size().unstack(fill_value=0)
quality['Total'] = quality.sum(axis=1)
print(quality)

print('\n' + '-'*80)
print('BY COUNTRY:')
country_quality = df.groupby(['country', 'reliability']).size().unstack(fill_value=0)
country_quality['Total'] = country_quality.sum(axis=1)
country_quality['Official %'] = (country_quality.get('OFFICIAL', 0) / country_quality['Total'] * 100).round(0)
print(country_quality.sort_values('Official %', ascending=False))

In [ ]:
# Visualization: Data quality
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Pie chart of overall reliability
reliability_counts = df['reliability'].value_counts()
colors_rel = {'OFFICIAL': '#27ae60', 'RESEARCH': '#f39c12', 'ESTIMATE': '#95a5a6'}
axes[0].pie(reliability_counts, labels=reliability_counts.index, autopct='%1.0f%%',
            colors=[colors_rel[x] for x in reliability_counts.index], startangle=90)
axes[0].set_title('Overall Data Reliability', fontsize=12, fontweight='bold')

# 2. Stacked bar by region
quality_pct = df.groupby(['region', 'reliability']).size().unstack(fill_value=0)
quality_pct = quality_pct.div(quality_pct.sum(axis=1), axis=0) * 100

quality_pct.plot(kind='bar', stacked=True, ax=axes[1], 
                  color=[colors_rel.get(c, '#999') for c in quality_pct.columns],
                  alpha=0.8)
axes[1].set_ylabel('Percentage')
axes[1].set_xlabel('Region')
axes[1].set_title('Data Quality by Region', fontsize=12, fontweight='bold')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend(title='Reliability')

plt.tight_layout()
plt.show()

print('\nNote: EU countries have 100% official Eurostat data')
print('      Balkan countries rely more on research estimates')

## 5. Time Trends Analysis

In [ ]:
# Time trends for Balkan countries (more historical data available)
print('='*80)
print('WAGE GAP TRENDS: BALKAN COUNTRIES')
print('='*80)

balkans_df = df[df['region'] == 'Balkans'].copy()

# Calculate yearly averages by country
yearly_balkans = balkans_df.groupby(['country', 'year'])['wage_gap_pct'].mean().unstack(level=0)
print(yearly_balkans.round(1))

In [ ]:
# Visualize trends
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Balkan countries trends
for country in BALKANS:
    country_data = balkans_df[balkans_df['country'] == country]
    yearly = country_data.groupby('year')['wage_gap_pct'].mean()
    if len(yearly) > 1:
        axes[0].plot(yearly.index, yearly.values, marker='o', linewidth=2, label=country)

axes[0].axhline(y=12.0, color='green', linestyle='--', alpha=0.5, label='EU Avg')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Wage Gap (%)')
axes[0].set_title('Wage Gap Trends: Balkan Countries', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. EU countries comparison (recent years)
eu_df = df[df['region'] == 'EU'].copy()
eu_recent = eu_df.groupby(['country', 'year'])['wage_gap_pct'].mean().unstack(level=0)

eu_recent.plot(kind='bar', ax=axes[1], width=0.8, alpha=0.8)
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Wage Gap (%)')
axes[1].set_title('Wage Gap: EU Countries (2021-2023)', fontsize=12, fontweight='bold')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()

## 6. Labor Market Context (World Bank Indicators)

In [ ]:
# Labor Force Participation analysis
print('='*80)
print('LABOR FORCE PARTICIPATION BY COUNTRY')
print('='*80)

# Calculate LFP gap
df['lfp_gap'] = df['SL.TLF.CACT.MA.ZS'] - df['SL.TLF.CACT.FE.ZS']

lfp_summary = df.groupby('country').agg({
    'SL.TLF.CACT.FE.ZS': 'mean',
    'SL.TLF.CACT.MA.ZS': 'mean',
    'lfp_gap': 'mean',
    'wage_gap_pct': 'mean',
    'region': 'first'
}).round(1)

lfp_summary.columns = ['Female LFP %', 'Male LFP %', 'LFP Gap (pp)', 'Wage Gap %', 'Region']
lfp_summary = lfp_summary.dropna().sort_values('LFP Gap (pp)', ascending=False)
print(lfp_summary)

In [ ]:
# LFP visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Filter to countries with LFP data
lfp_data = lfp_summary.dropna()

if len(lfp_data) > 0:
    # 1. LFP by gender
    x = np.arange(len(lfp_data))
    width = 0.35
    
    axes[0].bar(x - width/2, lfp_data['Female LFP %'], width, label='Female', color='#FF6B6B', alpha=0.8)
    axes[0].bar(x + width/2, lfp_data['Male LFP %'], width, label='Male', color='#4ECDC4', alpha=0.8)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(lfp_data.index, rotation=45, ha='right')
    axes[0].set_ylabel('Labor Force Participation (%)')
    axes[0].set_title('Labor Force Participation by Gender', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 2. Scatter: LFP gap vs Wage gap
    colors = ['#e74c3c' if r == 'Balkans' else '#3498db' for r in lfp_data['Region']]
    axes[1].scatter(lfp_data['LFP Gap (pp)'], lfp_data['Wage Gap %'], 
                    c=colors, s=100, alpha=0.7)
    for idx, row in lfp_data.iterrows():
        axes[1].annotate(idx, (row['LFP Gap (pp)'], row['Wage Gap %']),
                         textcoords='offset points', xytext=(5, 5), fontsize=9)
    axes[1].set_xlabel('Labor Force Participation Gap (pp)')
    axes[1].set_ylabel('Wage Gap (%)')
    axes[1].set_title('LFP Gap vs Wage Gap', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Correlation Analysis

In [ ]:
# Correlation analysis
print('='*80)
print('CORRELATION WITH WAGE GAP')
print('='*80)

# Select numeric World Bank columns
wb_cols = [col for col in df.columns if col.startswith('S') and col in WB_INDICATORS]

if len(wb_cols) > 0:
    correlations = df[['wage_gap_pct'] + wb_cols].corr()['wage_gap_pct'].drop('wage_gap_pct')
    correlations = correlations.dropna().sort_values()
    
    print('\nCorrelation with Wage Gap:')
    print('-'*60)
    for code, corr in correlations.items():
        name = WB_INDICATORS.get(code, code)[:35]
        strength = 'strong' if abs(corr) > 0.5 else 'moderate' if abs(corr) > 0.3 else 'weak'
        direction = '+' if corr > 0 else '-'
        print(f'{name:35} {corr:+.3f} ({strength})')

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))

# Select key columns
key_cols = ['wage_gap_pct']
for col in ['SL.TLF.CACT.FE.ZS', 'SL.TLF.CACT.MA.ZS', 'SL.UEM.TOTL.FE.ZS', 
            'SL.UEM.TOTL.MA.ZS', 'SE.TER.CUAT.BA.FE.ZS']:
    if col in df.columns:
        key_cols.append(col)

if len(key_cols) > 1:
    corr_matrix = df[key_cols].corr()
    
    # Rename for display
    display_names = {
        'wage_gap_pct': 'Wage Gap',
        'SL.TLF.CACT.FE.ZS': 'LFP Female',
        'SL.TLF.CACT.MA.ZS': 'LFP Male',
        'SL.UEM.TOTL.FE.ZS': 'Unemp Female',
        'SL.UEM.TOTL.MA.ZS': 'Unemp Male',
        'SE.TER.CUAT.BA.FE.ZS': 'Edu Female'
    }
    corr_matrix.index = [display_names.get(c, c) for c in corr_matrix.index]
    corr_matrix.columns = [display_names.get(c, c) for c in corr_matrix.columns]
    
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                ax=ax, linewidths=0.5, square=True, vmin=-1, vmax=1)
    ax.set_title('Correlation Matrix: Wage Gap & Labor Indicators', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Key Findings Summary

In [ ]:
print('='*80)
print('KEY FINDINGS: VALIDATED GENDER WAGE GAP ANALYSIS')
print('='*80)

print('\n1. REGIONAL DISPARITY')
print('-'*60)
balkans_avg = df[df['region']=='Balkans']['wage_gap_pct'].mean()
eu_avg = df[df['region']=='EU']['wage_gap_pct'].mean()
print(f'   Balkans average: {balkans_avg:.1f}%')
print(f'   EU average: {eu_avg:.1f}%')
print(f'   Gap: {balkans_avg - eu_avg:.1f} percentage points')
print(f'   Balkans gap is {(balkans_avg/eu_avg - 1)*100:.0f}% higher than EU')

print('\n2. COUNTRY HIGHLIGHTS')
print('-'*60)
print(f'   Highest gap: North Macedonia (17.8%) - Balkan')
print(f'   Lowest gap: Italy (3.0%) - EU')
print(f'   Hungary (17.4%) has EU\'s highest gap - similar to Balkans')

print('\n3. DATA QUALITY')
print('-'*60)
official_pct = len(df[df['reliability']=='OFFICIAL']) / len(df) * 100
print(f'   Official data: {official_pct:.0f}% of records')
print(f'   EU countries: 100% official (Eurostat)')
print(f'   Balkan countries: Mix of official/research/estimates')

print('\n4. LABOR MARKET CONTEXT')
print('-'*60)
nm_lfp_gap = df[df['country']=='North Macedonia']['lfp_gap'].mean()
print(f'   North Macedonia LFP gap: {nm_lfp_gap:.1f} pp (women participate less)')
print(f'   Wage gap correlates with labor force participation gap')

print('\n5. RECOMMENDATIONS')
print('-'*60)
print('   - Focus analysis on NM, Serbia, Montenegro (best Balkan data)')
print('   - Use EU countries as benchmark comparison')
print('   - Note data quality when presenting findings')
print('   - Hungary serves as EU outlier example (high gap like Balkans)')

print('\n' + '='*80)

In [ ]:
# Export summary table
summary_export = df.groupby('country').agg({
    'wage_gap_pct': ['mean', 'std'],
    'reliability': lambda x: x.mode()[0],
    'region': 'first',
    'year': ['min', 'max']
}).round(2)

summary_export.columns = ['Avg Gap %', 'Std Dev', 'Data Quality', 'Region', 'Year From', 'Year To']
summary_export = summary_export.sort_values('Avg Gap %', ascending=False)

summary_export.to_csv('../data/processed/country_summary_validated.csv')
print('Summary exported to: data/processed/country_summary_validated.csv')
print('\n')
print(summary_export)